### Modelo de Otimização Linear para Mega Sena 

Suponha que seu grupo foi contratado para fazer um modelo de otimização linear que encontre uma aposta de n números na mega-sena tal que:
- O(a) apostador(a) entra inicialmente com um número da sorte escolhido arbitrariamente por ele/ela;
- O modelo prioriza a escolha dos demais n-1 números menos sorteados considerando o histórico des sorteios até o dia 7 de abril de 2025 (dados anexos);
- A diferença entre a quantidade de números pares e a quantidade de números ímpares seja no máximo 2, considerando também o número da sorte;
- A diferença entre a quantidade de números na 1a metade superior e na 2a metade inferior do bilhete seja no máximo 2, considerando também o número da sorte;


In [1]:
## Bibliotecas
import pandas as pd
from amplpy import AMPL, ampl_notebook

data = pd.read_excel("data/Mega-Sena.xlsx")

# Eliminando as colunas irrelevantes para o problema
data_processado = data.drop(columns=['Concurso', 'Data do Sorteio', 'Ganhadores 6 acertos', 'Cidade / UF',
       'Rateio 6 acertos', 'Ganhadores 5 acertos', 'Rateio 5 acertos',
       'Ganhadores 4 acertos', 'Rateio 4 acertos', 'Acumulado 6 acertos',
       'Arrecadação Total', 'Estimativa prêmio',
       'Acumulado Sorteio Especial Mega da Virada', 'Observação'])

# Calculando as frequências de cada número 
frequencias = data_processado.stack().value_counts()

# Criando um arquivo .dat com as frequências
with open("data/frequencias.dat", "w") as f:
    f.write("param f :=\n")
    for valor, freq in frequencias.items():
        f.write(f"  {valor}   {freq}\n")
    f.write(";\n")

### Modelagem matemática do problema proposto, utilizando AMPL

In [2]:
def mega_sena_linear_model(n: int,
                           numero_sorte:int,
                           path_frequencias:str)-> list:
    """
    Função para otimização linear de um jogo na Mega-Sena
    baseado na frequências de resultados até 7 de abril de 2025.

    (O presente código é apenas um exercício didático, não recomendando
    inicitaiva nenhuma para apostas)

    Args:
        n (int): O tamanho da aposta
        numero_sorte (int): O número que você quer que esteja presente na aposta
        path_frequencias (str): O caminho para .dat que contém as frequências
    
    Return:
        aposta (List): Os números da aposta
    """
    if n > 60 or n < 0:
        raise ValueError("O número escolhido precisa estar entre 1 e 60")
    
    # Inicializa o ambiente do AMPL com a licença gratuita e o módulo HiGHS
    ampl_notebook(modules=["highs"], license_uuid="2e003210-ce0e-4ca9-bb4e-5d4a704b43a3")

    # Cria uma instância do AMPL
    ampl = AMPL()

    # Define o Modelo
    modelo = """ 

    # Variável de decisão
    var x {i in 1..60} binary; # Se o i-ésimo número está presente na aposta

    # Parâmetros 
    param n;                 # Tamanho da aposta
    param f {i in 1..60};    # Frequência do i-ésimo número 
    param s;                 # o número escolhido
    param k {i in 0..29};    # Controlar a paridade dos números

    # Função-objetivo
    minimize fobj: sum {i in 1..60} x[i]*f[i];

    # Restrições

    subject to tamanho_aposta: sum{i in 1..60} x[i] = n; 
    subject to s_precisa_constar: x[s] = 1;
    subject to paridade1: sum{i in 1..60: i mod 2 = 0} x[i] - sum{i in 1..60: i mod 2 != 0} x[i] <= 2;
    subject to paridade2: sum{i in 1..60: i mod 2 = 0} x[i] - sum{i in 1..60: i mod 2 != 0} x[i] >= -2;
    subject to diff1: sum{i in 1..30} x[i] - sum{i in 31..60} x[i] <= 2;
    subject to diff2: sum{i in 1..30} x[i] - sum{i in 31..60} x[i] >= -2;

    """

    # Carregar o modelo
    ampl.eval(modelo)

    # Carrega os dados do arquivo
    ampl.read_data(path_frequencias)
    ampl.param['n'] = n
    ampl.param['s'] = numero_sorte

    # Etapa 6: Define o solver gratuito HiGHS
    ampl.set_option('solver', 'highs')

    # Etapa 7: Resolve o modelo
    ampl.solve()

    # Recuperar a solução
    df1 = ampl.get_variable('x').get_values().to_pandas()

    aposta = []
    for idx, row in df1.iterrows():
        if row.iloc[0] == 1:
            aposta.append(idx)
            

    print("\nValor da função objetivo:")
    print(ampl.get_objective('fobj').value())

    print('Aposta->', sorted(aposta))

    return sorted(aposta)

### Testando a função criada

In [3]:
# Para 6 números
r = mega_sena_linear_model(n=6,
                       numero_sorte=10,
                       path_frequencias="data/frequencias.dat")

Licensed to AMPL Academic Community Edition License for <mangussi.arthur@unifesp.br>.
HiGHS 1.10.0HiGHS 1.10.0: optimal solution; objective 1561
2 simplex iterations
1 branching nodes

Valor da função objetivo:
1561.0
Aposta-> [10, 15, 21, 26, 48, 55]


In [4]:
# Para 7 números
r = mega_sena_linear_model(n=7,
                       numero_sorte=1,
                       path_frequencias="data/frequencias.dat")

Licensed to AMPL Academic Community Edition License for <mangussi.arthur@unifesp.br>.
HiGHS 1.10.0HiGHS 1.10.0: optimal solution; objective 1772
4 simplex iterations
1 branching nodes

Valor da função objetivo:
1772.0
Aposta-> [1, 15, 21, 26, 40, 48, 55]


In [5]:
# Para 8 números
r = mega_sena_linear_model(n=8,
                       numero_sorte=60,
                       path_frequencias="data/frequencias.dat")

Licensed to AMPL Academic Community Edition License for <mangussi.arthur@unifesp.br>.
HiGHS 1.10.0HiGHS 1.10.0: optimal solution; objective 2018
1 simplex iterations
1 branching nodes

Valor da função objetivo:
2018.0
Aposta-> [15, 21, 22, 26, 31, 48, 55, 60]


In [6]:
# Para 9 números
r = mega_sena_linear_model(n=9,
                       numero_sorte=15,
                       path_frequencias="data/frequencias.dat")

Licensed to AMPL Academic Community Edition License for <mangussi.arthur@unifesp.br>.
HiGHS 1.10.0HiGHS 1.10.0: optimal solution; objective 2276
1 simplex iterations
1 branching nodes

Valor da função objetivo:
2276.0
Aposta-> [3, 15, 21, 22, 26, 31, 40, 48, 55]


In [7]:
# Para 10 números
r = mega_sena_linear_model(n=10,
                       numero_sorte=2,
                       path_frequencias="data/frequencias.dat")

Licensed to AMPL Academic Community Edition License for <mangussi.arthur@unifesp.br>.
HiGHS 1.10.0HiGHS 1.10.0: optimal solution; objective 2559
1 simplex iterations
1 branching nodes

Valor da função objetivo:
2559.0
Aposta-> [2, 3, 15, 21, 22, 26, 31, 40, 48, 55]
